In [4]:
!pip install pyLDAvis
!pip install nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.2 MB/s eta 0:00:00


In [2]:
import zipfile
import pandas as pd
import os

# Upload the ZIP file first using Colab interface
uploaded_zip_path = "/content/arxiv_data.csv.zip"  # Path to uploaded ZIP file
extracted_path = "/content/extracted_arxiv"

# Extract the ZIP file
with zipfile.ZipFile(uploaded_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

# List extracted files
print("Extracted Files:", os.listdir(extracted_path))

# Load the CSV file
csv_file_path = os.path.join(extracted_path, 'arxiv_data.csv')
df = pd.read_csv(csv_file_path)

# Check the dataset structure
print("Columns in dataset:", df.columns)
print("Number of records:", df.shape[0])

# Display a sample of the dataset
df.head()

Extracted Files: ['arxiv_data.csv']
Columns in dataset: Index(['titles', 'summaries', 'terms'], dtype='object')
Number of records: 51774


,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
# Instead of importing pyLDAvis.sklearn directly, enable sklearn integration:
pyLDAvis.enable_notebook()  # Enable notebook display for pyLDAvis
import matplotlib.pyplot as plt

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Download the missing punkt_tab resource

# Drop rows with missing summaries
df = df.dropna(subset=['summaries'])

# Preprocessing function
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)  # Tokenize into words
    stop_words = set(stopwords.words('english'))  # Define stopwords
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove stopwords and non-alphabetic tokens
    return ' '.join(tokens)

# Apply preprocessing
df['cleaned_summaries'] = df['summaries'].apply(preprocess_text)

# Display cleaned data
df[['summaries', 'cleaned_summaries']].head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,summaries,cleaned_summaries
0,Stereo matching is one of the widely used tech...,stereo matching one widely used techniques inf...
1,The recent advancements in artificial intellig...,recent advancements artificial intelligence ai...
2,"In this paper, we proposed a novel mutual cons...",paper proposed novel mutual consistency networ...
3,Consistency training has proven to be an advan...,consistency training proven advanced framework...
4,"To ensure safety in automated driving, the cor...",ensure safety automated driving correct percep...


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Vectorize the cleaned summaries
vectorizer = CountVectorizer(max_df=0.95, min_df=10, stop_words='english')  # Adjust min_df for dataset size
doc_term_matrix = vectorizer.fit_transform(df['cleaned_summaries'])

# Train the LDA model
n_topics = 10  # Number of topics
lda_model = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda_model.fit(doc_term_matrix)

# Display the top words for each topic
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx + 1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

# Get feature names
feature_names = vectorizer.get_feature_names_out()

# Show the top 10 words per topic
display_topics(lda_model, feature_names, 10)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Topic 1:
data learning time models deep series model machine methods classification

Topic 2:
image images adversarial generative training model network face networks gan

Topic 3:
segmentation image images method proposed network semantic medical results based

Topic 4:
network neural networks performance attention convolutional model accuracy deep architecture

Topic 5:
point depth video method estimation scene network methods motion cloud

Topic 6:
learning reinforcement policy rl agent tasks algorithm algorithms agents environment

Topic 7:
model clustering models method using approach language algorithm explanations based

Topic 8:
graph learning model graphs representation information networks network representations neural

Topic 9:
data learning model algorithm problem models method distribution methods function

Topic 10:
object detection learning data performance training domain methods features propose


In [12]:
!pip install pyLDAvis==3.4.1 #install pyLDAvis version 3.4.1 or any version > 3.0
import pyLDAvis
# Instead of importing pyLDAvis.sklearn directly, enable sklearn integration:
pyLDAvis.enable_notebook()  # Enable notebook display for pyLDAvis
from IPython.display import HTML

# Calculate term frequencies from doc_term_matrix
term_frequency = doc_term_matrix.sum(axis=0).getA1()

# Prepare the visualization
lda_vis = pyLDAvis.prepare(
    topic_term_dists=lda_model.components_,
    doc_topic_dists=lda_model.transform(doc_term_matrix),
    doc_lengths=doc_term_matrix.sum(axis=1).getA1(),
    vocab=vectorizer.get_feature_names_out(),
    term_frequency=term_frequency,
    mds='pcoa'  # Use 'pcoa' instead of 'tsne' for stability
)

# Render inline visualization in Colab
HTML(pyLDAvis.prepared_data_to_html(lda_vis))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
